In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
import math
import torch
import torch.nn.functional as F
import llama_cpp

from path import *

In [108]:
phrase = "Italy"
k = 3
max_depth = 4

In [109]:
G = phrase_graph(phrase, k, max_depth)